# Supervised Learning -> Linear Regression

The first section (1 - 4) is identical across all regression models:
Keeping this part unchanged allows easy model comparison
and prevents pipeline-related mistakes.
Everything after that depends on the specific model.

1. Project setup and common pipeline
2. Dataset loading
3. Train-test split
4. Feature scaling (why we do it)
____
5. What is this model? (Intuition)
6. Model training
7. Model behavior and key hyperparameters
8. Predictions
9. Model evaluation
10. When to use it and when not to
11. Model persistence
12. Mathematical formulation (deep dive)
-----------------------------------------------------

## How this notebook should be read

This notebook is designed to be read **top to bottom**.

Before every code cell, you will find a short explanation describing:
- what we are about to do
- why this step is necessary
- how it fits into the overall process

The goal is not just to run the code,
but to understand what is happening at each step
and be able to adapt it to your own data.

-----------------------------------------------------
## What is KNN Regression? 

KNN Regression is a very different model compared to Linear Regression.

Instead of learning a global formula or a line,
KNN Regression makes predictions by looking at the data directly.

The idea is simple:
to predict the value for a new data point,
the model searches for the **K closest data points**
in the training set.

Once these neighbors are found:
- their target values are collected
- the prediction is computed as the **average** of those values

There is no explicit training phase where parameters are learned.
The model simply stores the training data
and uses it at prediction time.

For this reason, KNN is often described as a
**lazy, instance-based model**.

-----------------------------------------------------
## Why we start with intuition

Starting with intuition is especially important for KNN Regression.

All the model does is:
- measure distances between data points
- decide which points are close
- aggregate their target values

If this idea is clear,
the rest of the notebook becomes easy to follow.

Every step in the code will reflect this logic:
distance → neighbors → average → prediction.

-----------------------------------------------------

## What you should expect from the results

Before using KNN Regression, it is important to set expectations.

With KNN Regression, you should expect:
- predictions that adapt locally to the data
- good performance when similar data points exist nearby
- sensitivity to noise and outliers

This model often performs well as a flexible alternative
when linear assumptions are too restrictive,
but it may struggle on large datasets or high-dimensional data.


# Supervised Learning -> Linear Regression

The first section (1 - 4) is identical across all regression models:
Keeping this part unchanged allows easy model comparison
and prevents pipeline-related mistakes.
Everything after that depends on the specific model.

1. Project setup and common pipeline
2. Dataset loading
3. Train-test split
4. Feature scaling (why we do it)
____
5. What is this model? (Intuition)
6. Model training
7. Model parameters interpretation
8. Predictions
9. Model evaluation
10. When to use it and when not to
11. Model persistence
12. Mathematical formulation (deep dive)

-----------------------------------------------------

## How this notebook should be read

This notebook is designed to be read **top to bottom**.

Before every code cell, you will find a short explanation describing:
- what we are about to do
- why this step is necessary
- how it fits into the overall process

The goal is not just to run the code,
but to understand what is happening at each step
and be able to adapt it to your own data.

-----------------------------------------------------

## What is Linear Regression? 

Linear Regression is one of the simplest and most widely used machine learning models.

At a high level, Linear Regression tries to **draw a straight line through the data**
in such a way that the line is as close as possible to all data points.

Each data point represents:
- some input values (features)
- a real observed output (target)

The model:
- places a line (or a plane, or a higher-dimensional surface)
- measures how far each data point is from that line
- adjusts the line to minimize the overall distance to all points

These distances represent prediction errors.

If the line is too high:
- predictions are systematically too large

If the line is too low:
- predictions are systematically too small

By minimizing the total error, the model learns the **overall trend** in the data.

Once the line is learned, the model can:
- take new input values
- project them onto the learned line
- return a predicted output value

-----------------------------------------------------

## Why we start with intuition

Understanding the intuition first is important because:

- the code will directly reflect this idea
- every training step is just an automated way of adjusting the line
- every prediction is a projection onto that line

If this mental model is clear,
the rest of the notebook becomes much easier to follow.

-----------------------------------------------------

## What you should expect from the results

Before training the model, it is important to set expectations.

With Linear Regression, you should expect:
- reasonable but not perfect predictions
- a simple and interpretable model
- performance that captures general trends, not complex patterns

If the model performs poorly:
- it does not necessarily mean something is wrong
- it may simply mean the relationship is not linear

This model is often used as:
- a baseline
- a reference point for more complex models

____________________________________________


## 1. Project setup and common pipeline

In this section we prepare everything that is shared across all regression models.

The goal is to:
- set up a clean and reproducible environment
- import all common dependencies
- define a standard pipeline that will not change when switching models

This part of the notebook is intentionally kept simple and consistent.
If something changes here, it should change in all models.


### Why having a common pipeline matters

Using the same pipeline for all models allows us to:
- compare models fairly
- avoid data leakage
- reduce implementation errors
- focus on understanding the model instead of debugging the setup

From this point on, every model will start from the exact same data preparation steps.


In [ ]:
#   Used for numerical operations and data manipulation.
import numpy as np 
import pandas as pd

#  Provides a real-world regression dataset for demonstration purposes.
from sklearn.datasets import fetch_california_housing

#  Ensures a clean separation between training and test data.
from sklearn.model_selection import train_test_split

#  Applies feature scaling to ensure numerical stability and pipeline consistency.
from sklearn.preprocessing import StandardScaler

#  Used later to assess model performance.
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error

# Used for stabil path.
from pathlib import Path

#  Used to save trained models and preprocessing objects.
import joblib

# ____________________________________